https://lsjsj92.tistory.com/571

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.manifold import TSNE
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity
import os
import json
import random

##### 문서 임베딩(벡터화) 모델 함수화

In [ ]:
def make_doc2vec_models(tagged_data, tok, vector_size=128, window = 3, epochs = 40, min_count = 0, workers = 4):
    model = Doc2Vec(tagged_data, vector_size=vector_size, window=window, epochs=epochs, min_count=min_count, workers=workers)
    model.save(f'../datas/{tok}_news_model.doc2vec')

##### csv 파일 불러오는 함수

In [ ]:
def get_data(preprocess = True):
    if preprocess :
        data = pd.read_csv('../datas/preprocessing_tok_recipe_data1.csv' , encoding='cp949')
    else:
        user_1 = pd.read_csv('../datas/user_1.csv', header=None)
        user_2 = pd.read_csv('./datas/user_2.csv', header=None)
        user_3 = pd.read_csv('./datas/user_3.csv', header=None)

        columns = ['date', 'category', 'company', 'title', 'content', 'url']
        user_1.columns = columns
        user_2.columns = columns
        user_3.columns = columns

        data = pd.concat([user_1, user_2, user_3], axis = 0)
        data.reset_index(drop=True, inplace=True)

    return data

##### user_Id 숫자로 매핑

In [ ]:
def get_preprocessing_data(data):
#     data.drop(['date', 'company', 'url'], axis = 1, inplace =True)

    user_Id_mapping = {
    0 : 0,
    1 : 1,
    2 : 2
    }

    data['user_Id'] = data['user_Id'].map(user_Id_mapping)
    data['title_content'] = data['title'] + " " + data['content']

    data.drop(['title', 'content'], axis = 1, inplace = True)
    return data

In [ ]:
#def make_doc2vec_data(data, column, t_document=False):
#    data_doc = []
#    for tag, doc in zip(data.index, data[column]):
#       doc = doc.split(" ")
#        data_doc.append(([tag], doc))
#    if t_document:
#        data = [TaggedDocument(words=text, tags=tag) for tag, text in data_doc]
#        return data
#   else:
#        return data_doc

##### 콘텐츠(레시피) 추천 해주기

In [ ]:
def get_recommened_contents(user, data_doc, model):
    scores = []

    for tags, text in data_doc:
        trained_doc_vec = model.docvecs[tags[0]]
        scores.append(cosine_similarity(user.reshape(-1, 128), trained_doc_vec.reshape(-1, 128)))

    scores = np.array(scores).reshape(-1)
    scores = np.argsort(-scores)[:5]

    return data.loc[scores, :]

##### 유저 콘텐츠(레시피) 추천 임베딩(벡터화)

In [ ]:
def make_user_embedding(index_list, data_doc, model):
    user = []
    user_embedding = []
    for i in index_list:
        user.append(data_doc[i][0][0])
    for i in user:
        user_embedding.append(model.docvecs[i])
    user_embedding = np.array(user_embedding)
    user = np.mean(user_embedding, axis = 0)
    return user

#### Mecab

In [ ]:
!pip install eunjeon

from eunjeon import Mecab
import pandas as pd
import ast

def recipe_frm (yori_frm = True) :

  def change_token(sentence):
    tagger = Mecab()
    li = tagger.morphs(sentence)
    return ' '.join(s for s in li)

  yori_frm = pd.read_csv('../datas/preprocessing_tok_recipe_data.csv')
  yori_frm['title'] = yori_frm['title'].str.strip()

  yori_lst = list(yori_frm['content'].values)
  print(yori_lst)

  text = f"{yori_lst}"
  print(tagger.morphs(text))

  convert_lst = ast.literal_eval(change_token(text))
  type(convert_lst)

  yori_frm['mecab_tok'] = convert_lst
  yori_frm.to_csv('../datas/preprocessing_tok_recipe_data1.csv',header=True,index=False , encoding = "cp949")

##### 유저 히스토리 보여주는 함수

In [ ]:
def view_user_history(data):
    print(data[['user_Id', 'title_content']])

In [ ]:
data = get_data()

In [ ]:
data.head()

,user_Id,title,content,mecab_tok
0,0,딸기말차양갱,나머지 양갱물을 넣고 부은 후 냉장고에 2시간 정도 굳힌다. 냄비에 물과 한천을 넣...,나머지 양갱 물 을 넣 고 부 은 후 냉장고 에 2 시간 정도 굳힌다 . 냄비 에...
1,0,만두그라탕,"두부랑 햄도 올려주시고 만두는 한입 크기로 자르고 깻잎, 청양고추도 올려주세요 모짜...","두부 랑 햄 도 올려 주 시 고 만 두 는 한입 크기 로 자르 고 깻잎 , 청양 ..."
2,0,크림떡볶이,그 다음 떡을 넣고 잘 졸여줍니다 중약불 늘러 붙지 않도록 잘 저어주세요. 떡은 물...,그 다음 떡 을 넣 고 잘 졸여 줍니다 중약 불 늘러 붙 지 않 도록 잘 저 어 ...
3,0,치즈어니언링,기름을 넉넉히 붓고 양파를 앞뒤로 튀겨 주면 완성 입니다♡ 양파는 3등분 해서 동그...,기름 을 넉넉히 붓 고 양파 를 앞뒤 로 튀겨 주 면 완성 입니다 ♡ 양파 는 3...
4,0,충무김밥,김밥김 4등분해서 밥을 올려 말아 줍니다 냄비에 식초1맛술1을 넣어 살짝 삶아서 밥...,김밥 김 4 등분 해서 밥 을 올려 말 아 줍니다 냄비 에 식초 1 맛술 1 을 ...


##### 요리명과 레시피를 한 컬럼으로 결합

In [ ]:
get_preprocessing_data(data)

,user_Id,mecab_tok,title_content
0,0,나머지 양갱 물 을 넣 고 부 은 후 냉장고 에 2 시간 정도 굳힌다 . 냄비 에...,딸기말차양갱 나머지 양갱물을 넣고 부은 후 냉장고에 2시간 정도 굳힌다. 냄비에 물...
1,0,"두부 랑 햄 도 올려 주 시 고 만 두 는 한입 크기 로 자르 고 깻잎 , 청양 ...","만두그라탕 두부랑 햄도 올려주시고 만두는 한입 크기로 자르고 깻잎, 청양고추도 올려..."
2,0,그 다음 떡 을 넣 고 잘 졸여 줍니다 중약 불 늘러 붙 지 않 도록 잘 저 어 ...,크림떡볶이 그 다음 떡을 넣고 잘 졸여줍니다 중약불 늘러 붙지 않도록 잘 저어주세요...
3,0,기름 을 넉넉히 붓 고 양파 를 앞뒤 로 튀겨 주 면 완성 입니다 ♡ 양파 는 3...,치즈어니언링 기름을 넉넉히 붓고 양파를 앞뒤로 튀겨 주면 완성 입니다♡ 양파는 3등...
4,0,김밥 김 4 등분 해서 밥 을 올려 말 아 줍니다 냄비 에 식초 1 맛술 1 을 ...,충무김밥 김밥김 4등분해서 밥을 올려 말아 줍니다 냄비에 식초1맛술1을 넣어 살짝 ...
...,...,...,...
130,2,그 다음 모닝 빵 을 반 으로 가른 뒤 에 마요네즈 를 얇 게 발라 주 세요 ~ ...,미니새우버거 그 다음 모닝빵을 반으로 가른 뒤에 마요네즈를 얇게 발라주세요~마요네즈...
131,2,고구마 는 껍질 을 벗겨 삶 은 후 뜨거울 때 바로 으깨 주 세요 . 그라 탕 용...,고구마그라탕 고구마는 껍질을 벗겨 삶은후 뜨거울때 바로 으깨 주세요. 그라탕용기에 ...
132,2,1 시간 후 두 배 로 부푼 반죽 . 틀 의 80 % 정도 만 부풀 어 오르 면 ...,밤식빵 1시간후 두배로 부푼 반죽. 틀의 80% 정도만 부풀어 오르면 되요. 깍지를...
133,2,고 구마 를 깨끗 하 게 씻 어서 찜통 에 쪄 주 세요 . 통채 로 쪄도 되 고 ...,고구마보트 고구마를 깨끗하게 씻어서 찜통에 쪄주세요. 통채로 쪄도 되고 반을 잘라서...


In [ ]:
# data['mecab_tok'] = data['title_content']


In [ ]:
data

,user_Id,mecab_tok,title_content
0,0,나머지 양갱 물 을 넣 고 부 은 후 냉장고 에 2 시간 정도 굳힌다 . 냄비 에...,딸기말차양갱 나머지 양갱물을 넣고 부은 후 냉장고에 2시간 정도 굳힌다. 냄비에 물...
1,0,"두부 랑 햄 도 올려 주 시 고 만 두 는 한입 크기 로 자르 고 깻잎 , 청양 ...","만두그라탕 두부랑 햄도 올려주시고 만두는 한입 크기로 자르고 깻잎, 청양고추도 올려..."
2,0,그 다음 떡 을 넣 고 잘 졸여 줍니다 중약 불 늘러 붙 지 않 도록 잘 저 어 ...,크림떡볶이 그 다음 떡을 넣고 잘 졸여줍니다 중약불 늘러 붙지 않도록 잘 저어주세요...
3,0,기름 을 넉넉히 붓 고 양파 를 앞뒤 로 튀겨 주 면 완성 입니다 ♡ 양파 는 3...,치즈어니언링 기름을 넉넉히 붓고 양파를 앞뒤로 튀겨 주면 완성 입니다♡ 양파는 3등...
4,0,김밥 김 4 등분 해서 밥 을 올려 말 아 줍니다 냄비 에 식초 1 맛술 1 을 ...,충무김밥 김밥김 4등분해서 밥을 올려 말아 줍니다 냄비에 식초1맛술1을 넣어 살짝 ...
...,...,...,...
130,2,그 다음 모닝 빵 을 반 으로 가른 뒤 에 마요네즈 를 얇 게 발라 주 세요 ~ ...,미니새우버거 그 다음 모닝빵을 반으로 가른 뒤에 마요네즈를 얇게 발라주세요~마요네즈...
131,2,고구마 는 껍질 을 벗겨 삶 은 후 뜨거울 때 바로 으깨 주 세요 . 그라 탕 용...,고구마그라탕 고구마는 껍질을 벗겨 삶은후 뜨거울때 바로 으깨 주세요. 그라탕용기에 ...
132,2,1 시간 후 두 배 로 부푼 반죽 . 틀 의 80 % 정도 만 부풀 어 오르 면 ...,밤식빵 1시간후 두배로 부푼 반죽. 틀의 80% 정도만 부풀어 오르면 되요. 깍지를...
133,2,고 구마 를 깨끗 하 게 씻 어서 찜통 에 쪄 주 세요 . 통채 로 쪄도 되 고 ...,고구마보트 고구마를 깨끗하게 씻어서 찜통에 쪄주세요. 통채로 쪄도 되고 반을 잘라서...


##### 모델에 사용될 데이터 리스트화

In [ ]:
def make_doc2vec_data(data, column, t_document=False):
    data_doc = []
    for tag, doc in zip(data.index, data[column]):
        print(tag, ">>", doc)
        doc = doc.split(" ")
        data_doc.append(([tag], doc))
    if t_document:
        data = [TaggedDocument(words=text, tags=tag) for tag, text in data_doc]
        return data
    else:
        return data_doc

In [ ]:
data_doc_title_content_tag = make_doc2vec_data(data, 'title_content', t_document=True)
data_doc_title_content = make_doc2vec_data(data, 'title_content')
data_doc_tok_tag = make_doc2vec_data(data, 'mecab_tok', t_document=True)
data_doc_tok = make_doc2vec_data(data, 'mecab_tok')

0 >> 딸기말차양갱 나머지 양갱물을 넣고 부은 후 냉장고에 2시간 정도 굳힌다. 냄비에 물과 한천을 넣고 20~30분 불린 후 설탕을 넣고 중불에서 저으면서 끓인다. 랩을 씌운 사각무스틀에 양갱물을 약간 부은 후 딸기를 넣어 고정시킨다. 말차가루에 따뜻한 물을 넣고 섞은 후 2에 넣고 섞는다. 말차가루가 남을 경우 체에 한번 걸려주세요. 백앙금을 넣고 잘 섞으면서 끓인다. 적당한 크기에 맞춰 잘라 완성한다.
1 >> 만두그라탕 두부랑 햄도 올려주시고 만두는 한입 크기로 자르고 깻잎, 청양고추도 올려주세요 모짜렐라랑 체다치즈도 올립니다 양파와 햄, 양송이도 올려주세요. 잔자렌지 5분나눠서 돌려주세요 존비되 재료는 잘 썰어주세요 파스타 소스 올리고
2 >> 크림떡볶이 그 다음 떡을 넣고 잘 졸여줍니다 중약불 늘러 붙지 않도록 잘 저어주세요. 떡은 물에 씻어 물기를 빼주고 마늘은 편썰어주고 양파는 잘게 다져주세요. 베이컨들끼리 떨어질때 쯤 양파와 마늘을 넣고~ 식용유를 두른 뒤 베이컨을 볶아 줍니다. 양송이버섯이 윤기가 돌면 우유를 넣어주고 후추를 톡톡톡 뿌려 잘 섞어주세요. 양파가 살짝 투명해지면 양송이 버섯을 넣고 살짝 볶아주세요. 완성되면 파슬리가루 톡톡 뿌려주고~ 맛나게 먹어줍니다. 우유가 쫄아 살짝 걸쭉해 지면 체다치즈를 잘라 넣어주고 파마산치즈가루를 뿌려줍니다ㅎㅎ 잘 섞어준 뒤 내가 원하는 농도로 끓여주세요! 이때 양념을 맛보신 뒤 입맛에 맞게 소금을 살짝 넣어주세요.
3 >> 치즈어니언링 기름을 넉넉히 붓고 양파를 앞뒤로 튀겨 주면 완성 입니다♡ 양파는 3등분 해서 동그란 모양으로 분리해줍니다 큰양파링 안에 작은 양파링을 넣고 모짜렐라치즈를 그 사이에 끼워주세요 튀김가루,계란,빵가루 순으로 튀김옷을 입혀 주세요
4 >> 충무김밥 김밥김 4등분해서 밥을 올려 말아 줍니다 냄비에 식초1맛술1을 넣어 살짝 삶아서 밥에 깨 참기름 소금을 조금 넣어 살살 섞은 뒤 섞박지 레시피 @6949002 식은 후 양념장을 넣어 무쳐 둡니다 양념장을 만들고 어묵은 육수 2

# make doc2vec models

In [ ]:
make_doc2vec_models(data_doc_title_content_tag, tok=False)

In [ ]:
make_doc2vec_models(data_doc_tok_tag, tok=True)

# load doc2vec models

In [ ]:
model_title_content = Doc2Vec.load('../datas/False_news_model.doc2vec')

In [ ]:
model_tok = Doc2Vec.load('../datas/True_news_model.doc2vec')

#### user history

In [ ]:
user_category_1 = data.loc[random.sample(data.loc[data.user_Id == 0, :].index.values.tolist(), 5), :]  #경제
view_user_history(user_category_1)

    user_Id                                      title_content
5         0  가쓰오우동 건더기 & 액상소스 넣기 물 360ml (종이컵*2잔) 끓으면 버섯, 청...
9         0  고구마견과볼 가루류를 각 그릇에 소분하여 나눠줍니다. 견과류 2줌을 칼등으로 빻거나...
2         0  크림떡볶이 그 다음 떡을 넣고 잘 졸여줍니다 중약불 늘러 붙지 않도록 잘 저어주세요...
0         0  딸기말차양갱 나머지 양갱물을 넣고 부은 후 냉장고에 2시간 정도 굳힌다. 냄비에 물...
19        0  떡갈비밥버거 구워진 떡갈비를 올리고 양념한 밥을 또 올려줍니다. 떡갈비1장 그릇에 ...


In [ ]:
user_category_2 = data.loc[random.sample(data.loc[data.user_Id == 1, :].index.values.tolist(), 5), :]  #정치
view_user_history(user_category_2)

    user_Id                                      title_content
73        1  전부치는것 힘드시다고요~쉽고 빠르고 간편하게 전부치는법 계란을 3개 깨서 소금을 아...
71        1  집밥도시락)6가지반찬!! 삼치구이도시락^^ 계란물을 만들어놓고~ 계란물을 묻혀서 구...
50        1  햄버거 구워낸 빵 아랫부분 쪽에 마요네즈를 발라주세요. 노릇하게 구워진 패티를 양배...
43        1  만두피디저트 180℃로 예열한 오븐에 10분간 굽는다 다진 딸기를 넣고 잘 섞어 준...
67        1  고추튀김 1 + 2에 다진 돼지고기, 빵가루, 계란, 소금, 후추, 마늘, 맛술을 ...


In [ ]:
user_category_3 = data.loc[random.sample(data.loc[data.user_Id == 2, :].index.values.tolist(), 5), :]  #IT 과학
view_user_history(user_category_3)

     user_Id                                      title_content
90         2  오늘뭐먹지  성시경의 고추장찌개* 감자1개, 양파 1/2개, 애호박 1/2개, 당근...
123        2  베이컨김치말이밥 김치도 양파 볶은 팬에 기름을 두르지 말고 김치 물기만 날려줄 정도...
88         2  명절에 대표 음식 동그랑땡 새우젓이들어가서 맛있어요~~^^ 다진마늘 참기름 새우젖 ...
120        2  참치깻잎주먹밥 그 다음 끓는물에 참치를 넣어서 살짝 데쳐주신 다음에 물기를 빼주세요...
104        2  짜장만들기 / 짜장밥 만들기 간단해요 10분끓인후 짜장소스를 부어주고 약불에 걸죽해...


In [ ]:
user_1 = make_user_embedding(user_category_1.index.values.tolist(), data_doc_title_content, model_title_content) # 경제
user_2 = make_user_embedding(user_category_2.index.values.tolist(), data_doc_title_content, model_title_content) # 정치
user_3 = make_user_embedding(user_category_3.index.values.tolist(), data_doc_title_content, model_title_content) # IT과학

C:\Users\USER\AppData\Local\Temp\ipykernel_9080\3967141740.py:7: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  user_embedding.append(model.docvecs[i])


In [ ]:
result = get_recommened_contents(user_1, data_doc_title_content, model_title_content)
pd.DataFrame(result.loc[:, ['user_Id', 'title_content']])

C:\Users\USER\AppData\Local\Temp\ipykernel_9080\1388443086.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\USER\AppData\Local\Temp\ipykernel_9080\1388443086.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\USER\AppData\Local\Temp\ipykernel_9080\1388443086.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\USER\AppData\Local\Temp\ipykernel_9080\1388443086.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\USER\AppData\Local\Temp\ipykernel_9080\1388443086.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc

,user_Id,title_content
30,0,과일샌드위치 계란은 동그랗게 부쳐서 올려주세요 랩으로 두번정도 돌돌 말아 반으로 썰...
28,0,"콩나물제육볶음 *고기밑간 간장2,맛술2,마늘1, 후추 2톡톡 고기가 반쯤 익었을때 ..."
20,0,견과류멸치볶음주먹밥 @6951684 밥새우가 들어간 견과류멸치볶음 레시피네요 견과류...
49,1,메밀배추전 뒤집어준 배추잎에 홍고추 썰어놓은것을 올려주시고 그위에 메밀반죽을 조금씩...
45,1,양배추스프커리 국자로 슬슬 저어가며 끓이다가 약간 점도가 생기면 불을 끈다. 물 2...


In [ ]:
result = get_recommened_contents(user_2, data_doc_title_content, model_title_content)
pd.DataFrame(result.loc[:, ['user_Id', 'title_content']])

C:\Users\USER\AppData\Local\Temp\ipykernel_9080\1388443086.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\USER\AppData\Local\Temp\ipykernel_9080\1388443086.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\USER\AppData\Local\Temp\ipykernel_9080\1388443086.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\USER\AppData\Local\Temp\ipykernel_9080\1388443086.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\USER\AppData\Local\Temp\ipykernel_9080\1388443086.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc

,user_Id,title_content
50,1,햄버거 구워낸 빵 아랫부분 쪽에 마요네즈를 발라주세요. 노릇하게 구워진 패티를 양배...
84,2,불고기낙지전골 : 일명 불낙전골~ 깨끗히 씻어서 낙지를 잘라주세요. 낙지 2마리는 ...
20,0,견과류멸치볶음주먹밥 @6951684 밥새우가 들어간 견과류멸치볶음 레시피네요 견과류...
30,0,과일샌드위치 계란은 동그랗게 부쳐서 올려주세요 랩으로 두번정도 돌돌 말아 반으로 썰...
73,1,전부치는것 힘드시다고요~쉽고 빠르고 간편하게 전부치는법 계란을 3개 깨서 소금을 아...


In [ ]:
result = get_recommened_contents(user_3, data_doc_title_content, model_title_content)
pd.DataFrame(result.loc[:, ['user_Id', 'title_content']])

C:\Users\USER\AppData\Local\Temp\ipykernel_9080\1388443086.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\USER\AppData\Local\Temp\ipykernel_9080\1388443086.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\USER\AppData\Local\Temp\ipykernel_9080\1388443086.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\USER\AppData\Local\Temp\ipykernel_9080\1388443086.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\USER\AppData\Local\Temp\ipykernel_9080\1388443086.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc

,user_Id,title_content
104,2,짜장만들기 / 짜장밥 만들기 간단해요 10분끓인후 짜장소스를 부어주고 약불에 걸죽해...
35,0,"애호박볶음 불을 끄고 참기름을 1/3t정도만 두르면 애호박볶음 완성! 그래, 이맛이..."
30,0,과일샌드위치 계란은 동그랗게 부쳐서 올려주세요 랩으로 두번정도 돌돌 말아 반으로 썰...
45,1,양배추스프커리 국자로 슬슬 저어가며 끓이다가 약간 점도가 생기면 불을 끈다. 물 2...
114,2,표고버섯탕수 건표고를 따뜻한 물에 설탕을 1/2T 정도 넣고 불려주세요. 표고버섯이...


# 형태소 분석 후 결과

In [ ]:
user_1 = make_user_embedding(user_category_1.index.values.tolist(), data_doc_tok, model_tok) # 경제
user_2 = make_user_embedding(user_category_2.index.values.tolist(), data_doc_tok, model_tok) # 정치
user_3 = make_user_embedding(user_category_3.index.values.tolist(), data_doc_tok, model_tok) # IT과학

C:\Users\USER\AppData\Local\Temp\ipykernel_9080\3967141740.py:7: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  user_embedding.append(model.docvecs[i])


In [ ]:
result = get_recommened_contents(user_1, data_doc_tok, model_tok)
recommend_result1 = pd.DataFrame(result.loc[:, ['user_Id', 'title_content']])
print(recommend_result1)

    user_Id                                      title_content
43        1  만두피디저트 180℃로 예열한 오븐에 10분간 굽는다 다진 딸기를 넣고 잘 섞어 준...
63        1  모듬전 계란물을 입히고 노릇노릇 전을 부칩니다 고기에 양념을 합니다 꼬치를 만듭니다...
0         0  딸기말차양갱 나머지 양갱물을 넣고 부은 후 냉장고에 2시간 정도 굳힌다. 냄비에 물...
4         0  충무김밥 김밥김 4등분해서 밥을 올려 말아 줍니다 냄비에 식초1맛술1을 넣어 살짝 ...
27        0  허니버터고구마 가열된 후라이팬에 버터 한스푼을 넣고 녹여주세요 고구마를 넣고 잘 볶...


C:\Users\USER\AppData\Local\Temp\ipykernel_9080\1388443086.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\USER\AppData\Local\Temp\ipykernel_9080\1388443086.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\USER\AppData\Local\Temp\ipykernel_9080\1388443086.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\USER\AppData\Local\Temp\ipykernel_9080\1388443086.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\USER\AppData\Local\Temp\ipykernel_9080\1388443086.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc

In [ ]:
result = get_recommened_contents(user_2, data_doc_tok, model_tok)
recommend_result2 = pd.DataFrame(result.loc[:, ['user_Id', 'title_content']])
print(recommend_result2)

    user_Id                                      title_content
27        0  허니버터고구마 가열된 후라이팬에 버터 한스푼을 넣고 녹여주세요 고구마를 넣고 잘 볶...
43        1  만두피디저트 180℃로 예열한 오븐에 10분간 굽는다 다진 딸기를 넣고 잘 섞어 준...
71        1  집밥도시락)6가지반찬!! 삼치구이도시락^^ 계란물을 만들어놓고~ 계란물을 묻혀서 구...
63        1  모듬전 계란물을 입히고 노릇노릇 전을 부칩니다 고기에 양념을 합니다 꼬치를 만듭니다...
34        0  동태전 계란은 골고루 풀어 줍니다. 동태포에 밀가루 옷을 입혀 줍니다. 밀가루를 앞...


C:\Users\USER\AppData\Local\Temp\ipykernel_9080\1388443086.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\USER\AppData\Local\Temp\ipykernel_9080\1388443086.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\USER\AppData\Local\Temp\ipykernel_9080\1388443086.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\USER\AppData\Local\Temp\ipykernel_9080\1388443086.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\USER\AppData\Local\Temp\ipykernel_9080\1388443086.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc

In [ ]:
result = get_recommened_contents(user_3, data_doc_tok, model_tok)
recommend_result3 = pd.DataFrame(result.loc[:, ['user_Id', 'title_content']])
print(recommend_result3)

    user_Id                                      title_content
27        0  허니버터고구마 가열된 후라이팬에 버터 한스푼을 넣고 녹여주세요 고구마를 넣고 잘 볶...
71        1  집밥도시락)6가지반찬!! 삼치구이도시락^^ 계란물을 만들어놓고~ 계란물을 묻혀서 구...
43        1  만두피디저트 180℃로 예열한 오븐에 10분간 굽는다 다진 딸기를 넣고 잘 섞어 준...
63        1  모듬전 계란물을 입히고 노릇노릇 전을 부칩니다 고기에 양념을 합니다 꼬치를 만듭니다...
88        2  명절에 대표 음식 동그랑땡 새우젓이들어가서 맛있어요~~^^ 다진마늘 참기름 새우젖 ...


C:\Users\USER\AppData\Local\Temp\ipykernel_9080\1388443086.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\USER\AppData\Local\Temp\ipykernel_9080\1388443086.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\USER\AppData\Local\Temp\ipykernel_9080\1388443086.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\USER\AppData\Local\Temp\ipykernel_9080\1388443086.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\USER\AppData\Local\Temp\ipykernel_9080\1388443086.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc

##### 추천된 결과 title(요리명) 받아오기

In [ ]:
# index = rresult.index.to_list()
# print(index)



In [ ]:
# def recipe_title_lst(rs) :
#     index = rs.index.to_list()
#     recipe_lst = []
#     for idx in index:
#         li = rs['title_content'][idx].split(' ',maxsplit=1)[0]
#         recipe_lst.append(li)
#     return recipe_lst

In [ ]:
def recipe_title_original(rs) :
    df = get_data()
    index = rs.index.to_list()
    recipe_lst = []
    for idx in index:
        li = df['title'][idx]
        recipe_lst.append(li)
    return recipe_lst

In [ ]:
recipe_title_original(recommend_result1)

['만두피디저트', '모듬전', '딸기말차양갱', '충무김밥', '허니버터고구마']

In [ ]:
recipe_title_original(recommend_result2)

['허니버터고구마', '만두피디저트', '집밥도시락)6가지반찬!! 삼치구이도시락^^', '모듬전', '동태전']

In [ ]:
recipe_title_original(recommend_result3)

['허니버터고구마',
 '집밥도시락)6가지반찬!! 삼치구이도시락^^',
 '만두피디저트',
 '모듬전',
 '명절에 대표 음식 동그랑땡 새우젓이들어가서 맛있어요~~^^']

In [ ]:
# list 형태로 현재 폴더에 json 파일 뽑기
def recipe_title_json_lst(rjson_lst) :
    recipe_title = recipe_title_original(rjson_lst)
    with open('./recipe_title.json', 'w' , encoding= 'utf-8') as f:
         json.dump(recipe_title, f ,ensure_ascii=False , indent = 2)

# dict 형태로 현재 폴더에 json 파일뽑기
def recipe_title_json_dict(rjson_dict) :
    recipe_title = recipe_title_original(rjson_dict)
    recipe_title_dict = dict(zip(range(0, len(recipe_title) + 1), recipe_title))
    with open('./recipe_title.json', 'w' , encoding= 'utf-8') as f:
         json.dump(recipe_title_dict, f ,ensure_ascii=False , indent = 2)

위의 함수에 결과 변수 넣어주면 됨.

In [ ]:
recipe_title_json_lst(recommend_result1)

In [ ]:
# recipe_title_json_dict(recommend_result2)